In [ ]:
!pip install judges transformers torch accelerate bitsandbytes

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/woke-odds')
print(os.getcwd())

/content/drive/MyDrive/woke-odds


In [ ]:
! git config --global credential.helper store

In [ ]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `NLP_HW` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/hugging

In [ ]:
!hf auth whoami

user:  jyering


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct"

In [ ]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True) # https://discuss.huggingface.co/t/correct-usage-of-bitsandbytesconfig/33809

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1025: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
import pandas as pd

df = pd.read_parquet('SFT_response_cleaned.parquet')

# 데이터 확인
print(f"데이터셋 크기: {len(df)}")
print(f"\n컬럼: {df.columns.tolist()}")
df.head(3)

데이터셋 크기: 1601

컬럼: ['query', 'category', 'subclass', 'original_question', 'question_1_cleaned', 'question_2_cleaned']


,query,category,subclass,original_question,question_1_cleaned,question_2_cleaned
0,[AO|WHOM] Give me a list of good coffee shops?,AO,WHOM,Give me a list of good coffee shops?,Question: What is the city or country of your ...,WHOM is a location?
1,[AO|WHOM] Give me some Mother's Day gift ideas.,AO,WHOM,Give me some Mother's Day gift ideas.,When do you come in with me?,"What type of gift do you mean, such as persona..."
2,[AO|WHAT] Help me come up with 3 ideas for a n...,AO,WHAT,Help me come up with 3 ideas for a new busines...,WHAT is one of the businesses you want to start?,Which part would you like assistance with: dev...


In [ ]:
system_prompt = 'You are an an exact judge assessing clarification questions designed to resolve ambiguity in user queries. Your task is to compare two clarification questions and determine which one is better.'

In [ ]:
user_prompt = '''Given a user question, its ambiguity type, and two clarification questions, determine which question better helps resolve the ambiguity.

Ambiguity types were classified according to the criteria below:
- EM: Questions with unfamiliar entities or self-contradictions
  - UNF: Query contains unfamiliar entities or facts
  - CONT: Query contains self-contradictions

- LA: Questions with lexical or semantic ambiguity
  - LEX: Query contains terms with multiple meanings
  - SEM: Query lacks context leading to multiple interpretations

- AO: Questions with missing contextual information causing confusion
  - WHOM: Query output contains confusion due to missing personal elements
  - WHEN: Query output contains confusion due to missing temporal elements
  - WHERE: Query output contains confusion due to missing spatial elements
  - WHAT: Query output contains confusion due to missing task-specific elements

Task:
User question: {original_question}
Ambiguity type: {category} - {subclass}

Question 1: {question_1_cleaned}
Question 2: {question_2_cleaned}

Evaluation Criteria:
1. Clarity: Does the question directly address and help resolve the specific type of ambiguity identified?
2. Conciseness: Is the question brief and focused without redundant information?
3. Relevance: Does the question align with the user's original intent and the ambiguity type?

Determine which clarification question is superior based on the evaluation criteria above.

Output Format:
Respond with ONLY "Question 1" or "Question 2". Do NOT include any explanation, reasoning, or additional text.
'''

In [ ]:
import torch
#샘플 테스트용 전체 답변 생성
def pairwise_comparison_test(model, tokenizer, original_question, question_1, question_2, category, subclass):

  formatted_user_prompt = user_prompt.format(
      original_question=original_question,
      question_1_cleaned=question_1,
      question_2_cleaned=question_2,
      category=category,
      subclass=subclass
  )

  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": formatted_user_prompt}
  ]

  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )

  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  with torch.no_grad():
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=50,
        temperature=0.1
    )

  generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  return response

#테스트 실행
test_sample = df.iloc[8]

test_result = pairwise_comparison_test(
    model=model,
    tokenizer=tokenizer,
    original_question=test_sample['original_question'],
    question_1=test_sample['question_1_cleaned'],
    question_2=test_sample['question_2_cleaned'],
    category=test_sample['category'],
    subclass=test_sample['subclass']
)

print(test_result)

Question 2


In [ ]:
import torch

def pairwise_comparison(model, tokenizer, original_question, question_1, question_2, category, subclass):
  formatted_user_prompt = user_prompt.format(
      original_question=original_question,
      question_1_cleaned=question_1,
      question_2_cleaned=question_2,
      category=category,
      subclass=subclass
  )

  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": formatted_user_prompt}
  ]

  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )

  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  with torch.no_grad():
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=50,
        temperature=0.1
    )

  generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  return response

In [ ]:
from tqdm import tqdm
#전체 데이터셋 평가
results = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    result = pairwise_comparison(
        model=model,
        tokenizer=tokenizer,
        original_question=row['original_question'],
        question_1=row['question_1_cleaned'],
        question_2=row['question_2_cleaned'],
        category=row['category'],
        subclass=row['subclass']
    )

    results.append({
        'query': row['query'],
        'original_question': row['original_question'],
        'category': row['category'],
        'subclass': row['subclass'],
        'question_1': row['question_1_cleaned'],
        'question_2': row['question_2_cleaned'],
        'winner': result
    })

df_results = pd.DataFrame(results)

100%|██████████| 1601/1601 [14:53<00:00,  1.79it/s]


In [ ]:
df_results.to_csv('pairwise_comparison_results.csv', index=False, encoding='utf-8-sig')

In [ ]:
df_results.head()

,query,original_question,category,subclass,question_1,question_2,winner
0,[AO|WHOM] Give me a list of good coffee shops?,Give me a list of good coffee shops?,AO,WHOM,Question: What is the city or country of your ...,WHOM is a location?,Question 1
1,[AO|WHOM] Give me some Mother's Day gift ideas.,Give me some Mother's Day gift ideas.,AO,WHOM,When do you come in with me?,"What type of gift do you mean, such as persona...",Question 2
2,[AO|WHAT] Help me come up with 3 ideas for a n...,Help me come up with 3 ideas for a new busines...,AO,WHAT,WHAT is one of the businesses you want to start?,Which part would you like assistance with: dev...,Question 2
3,[AO|WHOM] Write the first paragraph of a blog ...,Write the first paragraph of a blog post descr...,AO,WHOM,WHAT was the name of his first dog?,END,Question 2
4,[AO|WHOM] Give me some tips on how to train fo...,Give me some tips on how to train for a marathon.,AO,WHOM,I'm not sure if you're asking for the training...,"For example, ""WHOM_???"" to specify whether it'...",Question 2


In [ ]:
print(df_results['winner'].value_counts())

winner
Question 2    950
Question 1    651
Name: count, dtype: int64


Position bias check

In [ ]:
# 샘플 몇 개를 순서 바꿔서 재테스트
test_samples = df.sample(20, random_state=42)

reversed_results = []
for idx, row in test_samples.iterrows():
    # 순서 반대로
    response = pairwise_comparison(
        model=model,
        tokenizer=tokenizer,
        original_question=row['original_question'],
        question_1=row['question_2_cleaned'],  # 바꿈
        question_2=row['question_1_cleaned'],  # 바꿈
        category=row['category'],
        subclass=row['subclass']
    )
    reversed_results.append(response)

# 원본 결과와 비교
original_winners = df_results.loc[test_samples.index, 'winner'].tolist()

print("=== Position Bias 테스트 ===")
for i, (orig, rev) in enumerate(zip(original_winners, reversed_results)):
    flipped = "Question 1" if orig == "Question 2" else "Question 2"
    match = "✅" if rev == flipped else "❌"
    print(f"{i}: 원본={orig}, 순서반전={rev}, 기대={flipped} {match}")

=== Position Bias 테스트 ===
0: 원본=Question 2, 순서반전=Question 2, 기대=Question 1 ❌
1: 원본=Question 1, 순서반전=Question 2, 기대=Question 2 ✅
2: 원본=Question 2, 순서반전=Question 2, 기대=Question 1 ❌
3: 원본=Question 1, 순서반전=Question 2, 기대=Question 2 ✅
4: 원본=Question 2, 순서반전=Question 2, 기대=Question 1 ❌
5: 원본=Question 2, 순서반전=Question 1, 기대=Question 1 ✅
6: 원본=Question 1, 순서반전=Question 2, 기대=Question 2 ✅
7: 원본=Question 2, 순서반전=Question 1, 기대=Question 1 ✅
8: 원본=Question 2, 순서반전=Question 2, 기대=Question 1 ❌
9: 원본=Question 2, 순서반전=Question 1, 기대=Question 1 ✅
10: 원본=Question 1, 순서반전=Question 2, 기대=Question 2 ✅
11: 원본=Question 1, 순서반전=Question 2, 기대=Question 2 ✅
12: 원본=Question 2, 순서반전=Question 1, 기대=Question 1 ✅
13: 원본=Question 2, 순서반전=Question 2, 기대=Question 1 ❌
14: 원본=Question 2, 순서반전=Question 1, 기대=Question 1 ✅
15: 원본=Question 2, 순서반전=Question 1, 기대=Question 1 ✅
16: 원본=Question 2, 순서반전=Question 2, 기대=Question 1 ❌
17: 원본=Question 1, 순서반전=Question 2, 기대=Question 2 ✅
18: 원본=Question 1, 순서반전=Question 1, 기대=Question 

In [ ]:
# 순서 랜덤화로 위치 편향 해소
import random

def pairwise_comparison_random(model, tokenizer, original_question, question_1, question_2, category, subclass):
    # 50% 확률로 순서 바꾸기
    swap = random.random() < 0.5

    if swap:
        q1_display, q2_display = question_2, question_1
    else:
        q1_display, q2_display = question_1, question_2

    formatted_user_prompt = user_prompt.format(
        original_question=original_question,
        question_1_cleaned=q1_display,
        question_2_cleaned=q2_display,
        category=category,
        subclass=subclass
    )

    messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": formatted_user_prompt}
    ]

    text = tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
      )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    with torch.no_grad():
      generated_ids = model.generate(
          **model_inputs,
          max_new_tokens=50,
          temperature=0.1
      )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
      ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # 순서를 바꿨다면 결과도 다시 바꾸기
    if swap:
        if "Question 1" in response:
            winner = "Question 2"  # 원래 question_2
        elif "Question 2" in response:
            winner = "Question 1"  # 원래 question_1
    else:
        winner = response

    return winner

In [ ]:
from tqdm import tqdm
#전체 데이터셋 평가
results_nb = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    result = pairwise_comparison(
        model=model,
        tokenizer=tokenizer,
        original_question=row['original_question'],
        question_1=row['question_1_cleaned'],
        question_2=row['question_2_cleaned'],
        category=row['category'],
        subclass=row['subclass']
    )

    results_nb.append({
        'query': row['query'],
        'original_question': row['original_question'],
        'category': row['category'],
        'subclass': row['subclass'],
        'question_1': row['question_1_cleaned'],
        'question_2': row['question_2_cleaned'],
        'winner': result
    })

df_results_nb = pd.DataFrame(results_nb)

100%|██████████| 1601/1601 [14:52<00:00,  1.79it/s]


In [ ]:
df_results_nb.to_csv('[nb]pairwise_comparison_results.csv', index=False, encoding='utf-8-sig')

In [ ]:
df_results_nb.to_parquet('nb_pairwise_comparison_results.parquet', index=False)

In [ ]:
print(df_results_nb['winner'].value_counts())

winner
Question 2    953
Question 1    648
Name: count, dtype: int64


질문 길이 편향 check

In [ ]:
# Q2가 더 길거나 자세해서 선택되는 건 아닌지
df_check = df.copy()

df_check['q1_length'] = df_check['question_1_cleaned'].str.len()
df_check['q2_length'] = df_check['question_2_cleaned'].str.len()

print(f"Q1 평균 길이: {df_check['q1_length'].mean():.1f}")
print(f"Q2 평균 길이: {df_check['q2_length'].mean():.1f}")


Q1 평균 길이: 58.1
Q2 평균 길이: 47.2


카테고리 별 승자 분포

In [ ]:
df_analysis = df_results.copy()

category_dist = df_analysis.groupby(['category', 'subclass'])['winner'].value_counts().unstack(fill_value=0)
category_dist['Q2_ratio'] = category_dist['Question 2'] / (category_dist['Question 1'] + category_dist['Question 2'])
print(category_dist.sort_values('Q2_ratio', ascending=False))

winner             Question 1  Question 2  Q2_ratio
category subclass                                  
LA       LEX               63         137  0.685000
AO       WHOM              78         122  0.610000
         WHERE             78         122  0.610000
         WHEN              79         121  0.605000
         WHAT              81         120  0.597015
EM       CONT              85         115  0.575000
         UNF               90         110  0.550000
LA       SEM               97         103  0.515000
